# Week 3 Assignment:

# Section 1
***
# Web Scraping to extract Toronto postal codes 

In this part, we are going to extract data from wikipedia ([See Website](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)).<br>
It contains Postal codes, Borough, Neighborhood of Toronto. In cases that data is available on a web, and there is no direct way to download it, web scraping  is a skill you can use to extract the data into a useful form that can be imported.

There are two solutions to scrap the web pages:
* Using Panda dataframe
* Using Beautiful Soup
<br>First of all, we need to import all required libraries.

In [1]:
# Uncomment below line to intall beautiful soup
# !pip install beautifulsoup4

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
import xml

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

We will use both methods to extract the data. 

## a) Extract data using Panda Dataframe

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
df_pd=tables[0]

df_pd.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The shape of data frame is as follows:

In [3]:
df_pd.shape

(288, 3)

## b) Extract data Using Beautiful Soap

In [4]:
# Input the URL using requests
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
# show the nested tags of the website using buatiful soap
soup = BeautifulSoup(url,'lxml') 

# Uncomment the below code to see the entire tags (huge output)
#soup  

We need to find the table in the results. As the result is huge, instead of present result, the section related to the table is shown in below figure. Based on the figure, the table is located under the class table of `wikitable`. 

<img src="Figures/NestedTag.JPG" height="1000" width="800">


We can use `.find` to access tags of `wikitable` class which is the data that we need. 

In [6]:
# Extract all contents of table 
class_='wikitable'
neighb = soup.find('table', class_ = class_)

Then, the `find_all()` method will be used to extract useful tags like < a > for hyperlinks, < table > for tables, < tr > for table rows, < th > for table headers, and < td > for table cells. In our case, we  will use 'tr' argument to show the table rows only.

In [7]:
# Print out the table rows 
neighb_rows = neighb.find_all('tr')

<img src="Figures/FindTr.JPG" height="800" width="600">

In this step, we need to clean and append all elements of data in a loop with `list` format.

In [8]:
# Remove empty string using split and append the elements
data = []
for row in neighb_rows:
    cleaned = row.text.split('\n')[1:-1]
    data.append(cleaned)    
data[0:10] 


[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

In the next step, Data will be converted to dataframe format.

In [9]:
df_bs= pd.DataFrame(data[1:], columns=data[0])
df_bs = df_bs.rename(columns={df_bs.columns[2]: "Neighborhood" })
df_bs.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


the shape generated dataframe: 

In [10]:
df_bs.shape

(288, 3)

So, we have extracted the required data from the web page. We can use one of generated dataframe by Panda or Beautiful soup for further analysis.

## Preparing dataset  
In this section we will clean and prepare data based on below rules:
* Process the cells that have an assigned borough and ignore cells with a borough that is Not assigned.
* Combine neighborhoods (separated with a comma) for a specific Postal Code
* Assign the neighborhood name the same as borough if the name of neighborhood is not assigned.

### 1) Process the cells that have an assigned borough and ignore cells with a borough that is Not assigned

In [11]:
df=df_bs

# Process the cells that have an assigned borough and ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']

# Reset index 
df.reset_index(inplace=True, drop=True)
df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### 2) Combine neighborhoods (separated with a comma) for a specific Postal Code

In [12]:
# Combine neighborhoods (separated with a comma) for a specific Postal Code
df_grouped=pd.DataFrame(df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))).reset_index()
df_grouped.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### 3) Assign the neighborhood name the same as borough if the name of neighborhood is not assigned

In [13]:
# Assign the neighborhood name the same as borough if the name of neighborhood is not assigned

# looking for rows with not assigned Neighborhood
NoNeigh_rows = df_grouped.Neighborhood == 'Not assigned'
df_grouped[NoNeigh_rows]


,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


There is only one row. So, we replace it with the borough name.

In [14]:
df_grouped.loc[NoNeigh_rows, 'Neighborhood'] = df_grouped.loc[NoNeigh_rows, 'Borough']
df_grouped.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


**The shape of prepared dataframe is as follows:**

In [15]:
print('The final dataframe shape: ',df_grouped.shape)

The final dataframe shape:  (103, 3)


# Section 2
---
#  Coordinates of each neighborhood
in this section, we need to get the latitude and the longitude coordinates of each neighborhood. There are two ways for getting the coordinates. You can use this data using `Geocoder` by uncomment the next section. As this process is very unstable, we will use data from the link __https://cocl.us/Geospatial_data__.

## a) Get Coordinates using Geocoder

In [16]:
## Uncomment all three next cells  to use Geocoder to gey the coordinates.

##! pip install geocoder
#import geocoder 

In [17]:
## Convert Postal code column to list
#pcodes = df_grouped['Postcode'].tolist()

In [18]:
## CAll for each postal code in data frame df_grouped
#for pcode in pcodes:

## initialize your variable to None
# lat_lng_coords = None

## loop until you get the coordinates
# while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(pcode))
#  lat_lng_coords = g.latlng
    
# df_grouped.loc[ df_grouped.Postcode == pcode, 'Latitude'] = lat_lng_coords[0]
# df_grouped.loc[ df_grouped.Postcode == pcode, 'Longitude'] = lat_lng_coords[1]

## b) Get Coordinates using Panda 

In [19]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Again, the loaded data should be investigated for any issues. As you can see the name of first column should be changed to `Postcode` to be consistent with `df_grouped` dataframe. 

In [20]:
# Change the name of column to Postcode 
geo_data = geo_data.rename(columns={geo_data.columns[0]: "Postcode" })

# Merge the dataframes
df_final=df_grouped.merge(geo_data, on = 'Postcode')
df_final.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [60]:
print('The size of Final data set Containing postcode, Borough, Neighborhood and Coordinates is ',df_final.shape)

The size of Final data set Containing postcode, Borough, Neighborhood and Coordinates is  (103, 5)


# Section 3
---
# Explore and cluster the neighborhoods in Toronto

Now, the data set is ready for further analysis which is exploring and clustering the dataset.

## Check the size of dataset

In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df_final['Borough'].unique()),df_final.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


## Get the latitude and longitude values of Toronto

In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods

In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Create a map of Toronto boroughs

In [25]:
# Generate lattitue and longtitude for each Borough 
BLL=[]
for boro in df_final['Borough'].unique():
        address = (boro + ", Ontario")
        geolocator = Nominatim(user_agent="to_explorer")
        
        loc = geolocator.geocode(address)
        lat = loc.latitude
        lon = loc.longitude
        boro_lat_lon=[boro, lat, lon]
        BLL.append(boro_lat_lon)

# Convert to dataframe        
Borough_Lat_lon=pd.DataFrame(BLL,columns=['Boro', 'Lat', 'Lon'])
Borough_Lat_lon["Lat"] = pd.to_numeric(Borough_Lat_lon["Lat"])
Borough_Lat_lon["Lon"] = pd.to_numeric(Borough_Lat_lon["Lon"])
Borough_Lat_lon    

,Boro,Lat,Lon
0,Scarborough,43.773077,-79.257774
1,North York,43.770817,-79.413300
2,East York,43.691339,-79.327821
3,East Toronto,43.626243,-79.396962
4,Central Toronto,43.644903,-79.381836
5,Downtown Toronto,43.655115,-79.380219
6,York,44.000752,-79.437222
7,West Toronto,43.644903,-79.381836
8,Queen's Park,43.659980,-79.390369
9,Mississauga,43.590338,-79.645729


In [26]:
# create a map for every Borough
map_allboro = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor in zip(Borough_Lat_lon ['Lat'], Borough_Lat_lon ['Lon'], Borough_Lat_lon ['Boro']):
    label = '{}'.format(bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_allboro)
        
map_allboro

## Target Area Exploration

We are going to explore North York borough.

In [91]:
Borough_target =['North York']
df_toronto = df_final[df_final['Borough'].isin(Borough_target)].reset_index(drop=True)
print('Number of Boroughs contain Toronto name: ',df_toronto.shape[0])
df_toronto.head()

Number of Boroughs contain Toronto name:  24


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


As we did with all of Toronto, let's visualize target area.

In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, post, borough, neigh in zip(df_toronto['Latitude'],
    df_toronto['Longitude'], df_toronto['Postcode'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Define Foursquare Credentials 

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [29]:

CLIENT_ID = 'DDKJFMDJVCQDOMJ5VQ2JN3X3LO3W2AQBG0ZZAVVYLOYCDDCF' # your Foursquare ID
CLIENT_SECRET = 'SQVPM0MWXJP5XVJO3UF45MZAZ3BPRO4LB5PJ21LNCKT4CAC3' # your Foursquare Secret
VERSION = '20190215'
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DDKJFMDJVCQDOMJ5VQ2JN3X3LO3W2AQBG0ZZAVVYLOYCDDCF
CLIENT_SECRET:SQVPM0MWXJP5XVJO3UF45MZAZ3BPRO4LB5PJ21LNCKT4CAC3


Let's explore the first neighborhood in our dataframe.¶

In [62]:
df_toronto.loc[0, 'Neighborhood']

'Hillcrest Village'

In [63]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


Get the top 100 venues within a radius of 500 meters.

In [64]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DDKJFMDJVCQDOMJ5VQ2JN3X3LO3W2AQBG0ZZAVVYLOYCDDCF&client_secret=SQVPM0MWXJP5XVJO3UF45MZAZ3BPRO4LB5PJ21LNCKT4CAC3&v=20190215&ll=43.8037622,-79.3634517&radius=500&limit=100'

In [65]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3e007022be1200320b864d'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

We know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [67]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,New York Fries,Fast Food Restaurant,43.803664,-79.363905
2,AY Jackson Pool,Pool,43.804515,-79.366138
3,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
4,Duncan Creek Park,Dog Run,43.805539,-79.360695
5,A.Y. Jackson Secondary School Track,Athletics & Sports,43.805068,-79.366677


It would good practice to visualize venues.  

In [68]:
map_ven = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long in zip(nearby_venues['lat'],
    nearby_venues['lng']):
    label = "{} - {}".format(lat, long)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ven)
    
map_ven

In [69]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


## Explore Neighborhoods

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [71]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


Let's check the size of the resulting dataframe

In [72]:
print(toronto_venues.shape)
toronto_venues.head()

(240, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


Let's check how many venues were returned for each neighborhood

In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,4,4,4,4,4,4
Downsview Central,3,3,3,3,3,3
Downsview Northwest,4,4,4,4,4,4
Downsview West,6,6,6,6,6,6
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [74]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 108 uniques categories.


## Analyze Each Neighborhood

In [75]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#neighb_colnum=toronto_onehot.columns.get_loc('Neighborhood')
# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-toronto_onehot.shape[1]+neighb_colnum]] + list(toronto_onehot.columns[:-toronto_onehot.shape[1]+neighb_colnum])
#toronto_onehot = toronto_onehot[fixed_columns]
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [76]:
toronto_onehot.shape

(240, 109)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.083333,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview, North Park, Upwood Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery, Humberlea",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [98]:
aa = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                             'Parrot', 'Parrot'],
                 'Max Speed': [380., 370., 24., 26.]})
aa_grouped = aa.groupby(['Animal']).mean().reset_index()
aa_grouped

,Animal,Max Speed
0,Falcon,375.0
1,Parrot,25.0


In [78]:
toronto_grouped.shape

(23, 109)

Let's print each neighborhood along with the top 10 most common venues

In [79]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                  venue  freq
0           Coffee Shop  0.11
1         Deli / Bodega  0.06
2         Shopping Mall  0.06
3  Fast Food Restaurant  0.06
4        Sandwich Place  0.06
5           Pizza Place  0.06
6           Supermarket  0.06
7           Bridal Shop  0.06
8            Restaurant  0.06
9      Sushi Restaurant  0.06


----Bayview Village----
                       venue  freq
0         Chinese Restaurant  0.25
1                       Café  0.25
2                       Bank  0.25
3        Japanese Restaurant  0.25
4          Accessories Store  0.00
5  Middle Eastern Restaurant  0.00
6                      Plaza  0.00
7                 Playground  0.00
8                Pizza Place  0.00
9                   Pharmacy  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1       Thai Restaurant  0.08
2           Coffee Shop  0.08
3         Grocery Store  0.04
4        

First, let's write a function to sort the venues in descending order

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Supermarket,Fast Food Restaurant,Diner,Deli / Bodega,Middle Eastern Restaurant,Pharmacy,Fried Chicken Joint,Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Thai Restaurant,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Restaurant,Pub,Butcher,Pizza Place,Café,Greek Restaurant
3,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Yoga Studio,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


## Cluster Neighborhoods
K-Means is a type of partitioning clustering which divides the data into K non-overlapping sphere-like clusters without any labels. It is relatively efficient on medium to large-sized data sets. This method tries to minimize intra-cluster distances and maximize the inter-cluster distances. To know how are the data points in a cluster, we need to use dissimilarity measures such as Euclidean distance or Average distance. The K means algorithm steps are as follow:

Randomly Initialize centroids (representative points of each cluster)
Distance calculation (Distance Matrix as a dissimilarity measure) to find the nearest centroid to data points
Assign each point to the nearest centroid
Update the centroids for every cluster
Repeat until there is no more change (move the centroids, calculate the distances from new centroids and assign data points to the nearest centroid) 

In [87]:
# set number of clusters
kclusters =5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, init='random',max_iter=3000,n_init=100).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([4, 2, 4, 3, 2, 1, 4, 4, 3, 1, 4, 4, 2, 4, 4, 4, 4, 3, 0, 4, 4, 4,
       3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [88]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4.0,Dog Run,Golf Course,Pool,Athletics & Sports,Fast Food Restaurant,Mediterranean Restaurant,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Toy / Game Store,Japanese Restaurant,Women's Store,Food Court,Electronics Store,Deli / Bodega
2,M2K,North York,Bayview Village,43.786947,-79.385975,2.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0.0,Cafeteria,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
toronto_merged.dropna(inplace=True)
toronto_merged.reset_index(inplace=True, drop=True)

temp = toronto_merged["Cluster Labels"].apply(lambda x: int(x) if x == x else "")
toronto_merged["Cluster Labels"]=temp
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Dog Run,Golf Course,Pool,Athletics & Sports,Fast Food Restaurant,Mediterranean Restaurant,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Toy / Game Store,Japanese Restaurant,Women's Store,Food Court,Electronics Store,Deli / Bodega
2,M2K,North York,Bayview Village,43.786947,-79.385975,2,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,Cafeteria,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2N,North York,Willowdale South,43.770120,-79.408493,4,Ramen Restaurant,Coffee Shop,Restaurant,Pizza Place,Sushi Restaurant,Sandwich Place,Café,Japanese Restaurant,Movie Theater,Indonesian Restaurant


 let's visualize the resulting clusters

In [90]:


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
In this section we will examine all clusters.

### Cluster 1

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,0,Cafeteria,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


### Cluster 2

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,1,Home Service,Food Truck,Baseball Field,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
22,North York,1,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


### Cluster 3

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,2,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
8,North York,2,Caribbean Restaurant,Gym / Fitness Center,Café,Japanese Restaurant,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
19,North York,2,Playground,Pub,Bakery,Japanese Restaurant,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


### Cluster 4

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,3,Park,Convenience Store,Bank,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
7,North York,3,Park,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
12,North York,3,Airport,Park,Bus Stop,Yoga Studio,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
20,North York,3,Construction & Landscaping,Park,Bakery,Basketball Court,Yoga Studio,Dog Run,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega


### Cluster 5

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Dog Run,Golf Course,Pool,Athletics & Sports,Fast Food Restaurant,Mediterranean Restaurant,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,North York,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Toy / Game Store,Japanese Restaurant,Women's Store,Food Court,Electronics Store,Deli / Bodega
4,North York,4,Ramen Restaurant,Coffee Shop,Restaurant,Pizza Place,Sushi Restaurant,Sandwich Place,Café,Japanese Restaurant,Movie Theater,Indonesian Restaurant
6,North York,4,Pizza Place,Discount Store,Pharmacy,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
9,North York,4,Coffee Shop,Gym,Beer Store,Bike Shop,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Restaurant,Japanese Restaurant,Sandwich Place
10,North York,4,Coffee Shop,Pizza Place,Supermarket,Fast Food Restaurant,Diner,Deli / Bodega,Middle Eastern Restaurant,Pharmacy,Fried Chicken Joint,Restaurant
11,North York,4,Coffee Shop,Miscellaneous Shop,Bar,Massage Studio,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
13,North York,4,Grocery Store,Park,Bank,Hotel,Shopping Mall,Yoga Studio,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
15,North York,4,Liquor Store,Grocery Store,Athletics & Sports,Discount Store,Yoga Studio,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
16,North York,4,Pizza Place,Portuguese Restaurant,Coffee Shop,Hockey Arena,Financial or Legal Service,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
